In [1]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pickle
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from html.parser import HTMLParser

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [2]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [3]:
def readWord2vecDict(file_dir):
    with open(file_dir, 'rb') as f:
        word2vecDict = pickle.load(f)
    f.close()
    return word2vecDict


en_word2vecDict = readWord2vecDict("en_dict_512.bin")
zh_word2vecDict = readWord2vecDict("zh_dict_512.bin")

# print(en_word2vecDict["premiumization"])
# print(zh_word2vecDict["我"])

In [4]:
# ls = list(zh_word2vecDict.values())

# for key in zh_word2vecDict.keys():
#     zh_word2vecDict[key] = zh_word2vecDict[key] * 0.01
# ls = list(zh_word2vecDict.values())
# # ls = np.array(ls)
# min_=1
# max_=-1
# for i in range(len(ls)):
#     if min(ls[i]) < min_:
#         min_ = min(ls[i])
#     if max(ls[i]) > max_:
#         max_ = max(ls[i])
# print("after:")
# print(min_, max_)
    
    

# for key in en_word2vecDict.keys():
#     en_word2vecDict[key] = en_word2vecDict[key] * 0.01
# ls = list(en_word2vecDict.values())
# # ls = np.array(ls)
# min_=1
# max_=-1
# for i in range(len(ls)):
#     if min(ls[i]) < min_:
#         min_ = min(ls[i])
#     if max(ls[i]) > max_:
#         max_ = max(ls[i])
# print("before")
# print(min_, max_)

In [5]:
en_word2vecDict["SOS"] = -np.ones(512)
en_word2vecDict["UNK"] = np.zeros(512)
en_word2vecDict["EOS"] = np.ones(512)
zh_word2vecDict["SOS"] = -np.ones(512)
zh_word2vecDict["UNK"] = np.zeros(512)
zh_word2vecDict["EOS"] = np.ones(512)

In [6]:
SOS_token = -1
EOS_token = 1

class Lang:
    def __init__(self, name, word2vecDict):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {-1: "SOS", 0:"UNK", 1:"EOS"}
        self.n_words = 3  # Count SOS, EOS
        self.word2vecDict = word2vecDict

    def addSentence(self, sentence):
        for word in sentence.strip().split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word in self.word2vecDict:
            if word not in self.word2index:
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
            else:
                self.word2count[word] += 1

In [7]:
class read_Langs(object):
    def __init__(self,dir):
        self.dir = dir
    def __iter__(self):
        for line in open(self.dir, encoding = 'utf-8'):
            if len(line.strip().split('\t'))==2:
                yield line.strip().split('\t')
            else:
                pass
def readLangs():
    pairs = read_Langs('train_pair')
    pairs = [pair for pair in pairs]
    input_lang = Lang("en", en_word2vecDict)
    output_lang = Lang("zh", zh_word2vecDict)
    return input_lang, output_lang, pairs

In [8]:
MAX_LENGTH = 20

def filterPair(p):
#     print(str(p[0]) + str(p[1]), end = "\r")
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData():
    input_lang, output_lang, pairs = readLangs()
    pairs = filterPairs(pairs)
    print("Read %s sentence pairs" % len(pairs))
#     i=0
    for pair in pairs:
#         print(i)
#         i+=1
#         print(pair)
        input_lang.addSentence(pair[0].strip())
        output_lang.addSentence(pair[1].strip())
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    print("Count of sentence pairs:")
    print(len(pairs))
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData()



Read 8774549 sentence pairs
Counted words:
en 97734
zh 128243
Count of sentence pairs:
8774549


In [10]:
# with open("input_lang.lang", 'wb') as f:
#     word2vecDict = pickle.dump(input_lang, f)
# f.close()
# with open("output_lang.lang", 'wb') as f:
#     word2vecDict = pickle.dump(output_lang, f)
# f.close()
# with open("pairs.lang", 'wb') as f:
#     word2vecDict = pickle.dump(pairs, f)
# f.close()
        
# with open("input_lang.lang", 'rb') as f:
#     input_lang = pickle.load(f)
# f.close()
# with open("output_lang.lang", 'rb') as f:
#     output_lang = pickle.load(f)
# f.close()
# with open("pairs.lang", 'rb') as f:
#     pairs = pickle.load(f)
# f.close()

In [11]:
def embedding(input, lang, word2vecDict):  #input(1, seq_len)
#     print(input)
    input = input.cpu().data.numpy()
#         print("input[i][0]")
#         print(input[i][0])
#         print("lang.index2word[input[i][0]]")
#         print(lang.index2word[input[i][0]])
    embeded = torch.FloatTensor(1, len(input[0]), 512)
    for i in range(len(input[0])):
        word = lang.index2word[input[0][i]]
        if word in word2vecDict:
            embeded[0, i] = torch.FloatTensor(word2vecDict[word])
        else:
            embeded[0, i] = torch.FloatTensor(word2vecDict["UNK"])
#         print("embedding[i]")
#         print(embedding[i])
    return Variable(embeded)  #embedding(1, seq_len, embedding_size)

In [12]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else 0 for word in sentence.strip().split(' ')]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.insert(0, SOS_token)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes))
    return result.cuda() if use_cuda else result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [13]:
def grad_multiply(x, scale):
    return GradMultiply.apply(x, scale)


class GradMultiply(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, scale):
        ctx.scale = scale
        res = x.new(x)
        ctx.mark_shared_storage((x, res))
        return res

    @staticmethod
    def backward(ctx, grad):
        return grad * ctx.scale, None

In [14]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.embedding_size = 512
        self.hidden_size = 512
        self.n_layers = 13
        self.linear1 = nn.Linear(self.embedding_size, self.hidden_size)
#         self.linear2 = nn.Linear(self.hidden_size, 2*self.hidden_size)
#         self.linear3 = nn.Linear(2*self.hidden_size, 2*self.hidden_size)
        self.out_channels = 2*self.hidden_size
        self.kernel = (3, self.hidden_size)
        self.padding = (int((self.kernel[0]-1)/2), 0)
        #conv input(N_batches, Channels_in, Height_in, Width_in)
        self.conv = nn.Conv2d(1, self.out_channels, self.kernel, (1,1), self.padding)
        
        
    def forward(self, input):  #input is only a sequence of indexes of words
        input = input[1:input.size(0)]  #delete the SOS_token
        input = input.unsqueeze(0)  #in input(seq_len), out input(1, seq_len) where 1 means batch size is 1
        global en_word2vecDict, input_lang
        input = embedding(input, input_lang, en_word2vecDict).cuda()*0.1  #out input(1, seq_len, embedding_size)
        input = F.dropout(input, p=0.1, training=self.training)
        input = input.unsqueeze(1) #out input(1, 1, seq_len, hidden_size)
        input_temp = input
#         input = self.linear1(input)  #out input(1, 1, seq_len, hidden_size)
        for _ in range(self.n_layers):
            input_ = input
            input = F.dropout(input, p=0.1, training=self.training)
            input = self.conv(input).transpose(1,3)  #out input(1, 1, seq_len, 2*hidden_size) 
            input = F.glu(input, 3)  #out input(1, 1, seq_len, hidden_size)
            input = (input + input_)*math.sqrt(0.5)
#             input = F.relu(input)
#         attn = input
#         out = input + input_temp
        input = grad_multiply(input, 1.0 / (2.0 * self.n_layers))  # scale gradients (this only affects backward, not forward)
        return input, input + input_temp

In [15]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.embedding_size = 512
        self.hidden_size = 512
        global output_lang
        self.dict_size = output_lang.n_words
        print(self.dict_size)
        self.n_layers = 13
        self.linear1 = nn.Linear(self.embedding_size, self.hidden_size)
        self.linear2 = nn.Linear(self.hidden_size, 2*self.hidden_size)
        self.linear3 = nn.Linear(self.hidden_size, self.dict_size)
        self.linear4 = nn.Linear(2*self.hidden_size, 2*self.hidden_size)
        self.in_channels = 1
        self.out_channels = 2*self.hidden_size
        self.kernel = (3, self.hidden_size)
        self.stride = (1, 1)
        self.padding = (0, 0)  #mannually pad, without using its inner auto padding
        self.conv = nn.Conv2d(self.in_channels, self.out_channels, self.kernel, self.stride, self.padding)
        
        
    def forward(self, target, enc_attn, enc_out):  #target is only a sequence of indexes of words
        input = target[0:-1]  #delete the EOS_token
        input = input.unsqueeze(0)  #target(tar_seq_len), input(1, tar_seq_len) where 1 means batch size is 1
        global zh_word2vecDict, output_lang
        input = embedding(input, output_lang, zh_word2vecDict).cuda()*0.1 #out input(1, tar_seq_len, embedding_size)
        input = F.dropout(input, p=0.1, training=self.training)
        input = input.unsqueeze(1) #out input(1, 1, tar_seq_len, hidden_size)
        input_temp = input
#         input = self.linear1(input)  #out input(1, 1, tar_seq_len, hidden_size)
        for _ in range(self.n_layers):
            input_ = input
            input = F.dropout(input, p=0.1, training=self.training)
            cat_temp = Variable(torch.FloatTensor(torch.zeros(1, 1, self.kernel[0]-1, self.hidden_size))).cuda()
            input = torch.cat((cat_temp, input), 2)  #pad left with kernel_size-1 elements
            input = self.conv(input)  #out input(1, 2*hidden_size, tar_seq_len, 1) 
                                        #where the second 1 means 2*hidden_size changes into is 1
            input = input.transpose(1, 3)  #out input(1, 1, tar_seq_len, 2*hidden_size)
            input = F.glu(input, 3)  #out input(1, 1, tar_seq_len, hidden_size)
            input__ = input
            input = (input + input_temp)*math.sqrt(0.5)
            input = torch.bmm(input.squeeze(0), enc_attn.squeeze(0).transpose(1, 2))  #out input(1, tar_seq_len, sor_seq_len)
            #softmax:result_i_j = exp(x_i_j)/sum(exp(x_j)), input should be in two directions
            input = F.softmax(input.squeeze(0)).unsqueeze(0) #out input(1, tar_seq_len, sor_seq_len)
            input = torch.bmm(input, enc_out.squeeze(0))  #out input(1, tar_seq_len, hidden_size)
            input = input*(enc_out.size(2) * math.sqrt(1.0/enc_out.size(2)))
            input = (input.unsqueeze(0) + input__)*math.sqrt(0.5) #out input(1, 1, tar_seq_len, hidden_size)
            input = (input + input_)*math.sqrt(0.5)
#             input = F.relu(input)
        input = input.squeeze(0).squeeze(0)  #out input(tar_seq_len, hidden_size)
        input = F.dropout(input, p=0.1, training=self.training)
        input = F.log_softmax(self.linear3(input))  #out input(tar_seq_len, dict_size) has tar_seq_len choices of words
        return input

In [16]:
class NMTModel(nn.Module):
    """
    NMTModel:
    Input:
        encoder:
        decoder:
        attention:
        generator:
    return:
    """

    def __init__(self, encoder, decocer):
        super(NMTModel, self).__init__()
        self.encoder = encoder
        self.decocer = decocer

    def forward(self, source, target):
        # attn(1, 1, seq_len, 2*hidden_size)
        # out(1, 1, seq_len, 2*hidden_size)
        attn, source_seq_out = self.encoder(source)

        # out(tar_seq_len, dict_size)
        out = self.decocer(target, attn, source_seq_out)

        return out

In [17]:
def train(input_variable, target_variable, NMTModel, NMTModel_optimizer, criterion):

    NMTModel_optimizer.zero_grad()
   
    loss = 0
    
    for i in range(len(input_variable)):
        out = NMTModel(input_variable[i], target_variable[i])
#         print("out")
#         print(out)
#         print("target_to_compare")
#         print(target_variable[i][1:target_variable[i].size(0)])
        loss += criterion(out, target_variable[i][1:target_variable[i].size(0)])
    loss.backward()
    
    NMTModel_optimizer.step()

    return loss.data[0]/(len(input_variable))

In [18]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [20]:
def saveModel(model, model_path):
    torch.save(model.state_dict(), model_path)

def loadModel(model, model_path):
    model.load_state_dict(torch.load(model_path))

In [21]:
def trainIters(NMTModel, n_iters, print_every=1000, plot_every=100, save_every = 5000, learning_rate=0.01):
    NMTModel.train(True)
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    NMTModel_optimizer = optim.SGD(NMTModel.parameters(), lr=learning_rate, momentum=0.99,nesterov=True)
    
    criterion = nn.NLLLoss()
    
    batch_size = 64
    
    for iter in range(1, n_iters+1):
        training_pairs = [variablesFromPair(random.choice(pairs)) for _ in range(batch_size)]
        input_variable_batch = [training_pairs[i][0] for i in range(batch_size)]
        target_variable_batch = [training_pairs[i][1] for i in range(batch_size)]
        
        loss = train(input_variable_batch, target_variable_batch, NMTModel, NMTModel_optimizer, criterion)
    
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print("%s  (%d  %d%%)  %f" %(timeSince(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        if iter % save_every == 0:
            saveModel(NMTModel, "./NMTModel_" + str(time.strftime('%d-%H',time.localtime(time.time()))) + ".mod")
    showPlot(plot_losses)

In [22]:
def weight_init(model):
    print(model.parameters())
    for param in model.parameters():
        print(type(param.data))
        torch.nn.init.normal(param.data, 0, 0.01)
#     print(model.weight)
    #torch.nn.init.normal(model.weight.data, mean=0, std=0.5)

In [23]:
encoder = Encoder()
decoder = Decoder()
nmtmodel = NMTModel(encoder, decoder)
# weight_init(nmtmodel)

if True:
    
    loadModel(nmtmodel, "./NMTModel_23-04.mod")

    
if use_cuda:
    nmtmodel = nmtmodel.cuda()

128243


In [24]:
# trainIters(nmtmodel, n_iters = 40000, print_every=100, plot_every = 10, save_every = 5000, learning_rate = 0.0001)

In [25]:
def evaluate(model, input_sentence):
    model.train(False)
    output_sentence = ""
    pair = [input_sentence, output_sentence]
    input_variable, _= variablesFromPair(pair)
    output_variable = Variable(torch.LongTensor([-1,1])).cuda()
    end_flag = False
    while(end_flag == False):
#         print(input_variable)
#         print(output_variable)
        out = model(input_variable, output_variable)
        topi = out.data.topk(1, dim = 1)
        indexes = topi[1].transpose(0,1).cpu().numpy()[0]
#         print(indexes)
        output_sentence = ""
        for i in range(len(indexes)):
            if i != len(indexes)-1 :
#                 print(str(i) + ": "  + str(output_lang.index2word[indexes[i]]))
                output_sentence += output_lang.index2word[indexes[i]] + " "
            else:
#                 print(str(i) + ": "  + str(output_lang.index2word[indexes[i]]))
                if output_lang.index2word[indexes[i]] == "EOS":
                    end_flag = True
                else:
                    output_sentence += output_lang.index2word[indexes[i]]
        pair = [input_sentence, output_sentence]
        input_variable, output_variable= variablesFromPair(pair)
    print(output_sentence)
    return output_sentence
#     print(topi)
#     print(indexes)

In [47]:
input_sentence = "For Access To Guest the floors 23 and 24 , Use Key Card Is Located on floor 24 ."
evaluate(nmtmodel, input_sentence.lower())

提供 UNK 的 选项 ， 使用 UNK ， 使用 钥匙 ， 使用 UNK UNK 的 位置 。 


'提供 UNK 的 选项 ， 使用 UNK ， 使用 钥匙 ， 使用 UNK UNK 的 位置 。 '

In [27]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.sentence_dict = {}
        self.getdataflag = False
        self.sentence_dict_key = ""

    def handle_starttag(self, tag, attrs):
        if tag == "seg":
            self.getdataflag = True
            self.sentence_dict_key = attrs[0][1]
            #print(attrs)
            
    def handle_data(self, data):
        if self.getdataflag == True:
            self.sentence_dict[self.sentence_dict_key] = data
        self.getdataflag = False

In [28]:
sgmfile = open("test_b.sgm",'r')
text = sgmfile.read()
parser = MyHTMLParser()
parser.feed(text)
en_sentences_dict = parser.sentence_dict.copy()

print(en_sentences_dict["999"])

 What am I going to do? - you'll figure it out. 


In [29]:
def normalizeString(s):
    #s=jieba.cut(s)
    #正则表达式，标点符号前加空格 去掉特殊标点符号 
    s = re.sub(r"(\,)", r" \1 ", s)
    s = re.sub(r"(\.)", r" \1 ", s)
    s = re.sub(r"(\?)", r" \1 ", s)
    s = re.sub(r"(\!)", r" \1 ", s)
    s = re.sub(r"(\:)", r" \1 ", s)

#     s = re.sub(r"([,!?])", r" \1", s)
#     s = re.sub(r"(\d),(\d)",r"\1,\2", s)
    s = re.sub(r"[^\da-zA-Z.!,'?]+", r" ", s)
    #s = re.sub(r"[a-zA-Z] - [a-zA-Z]", r"\1-\2 ", s)
    return s.lower().strip()
ss = "It's (df) df.,fdf@ !"
print(normalizeString(ss))

it's df df . , fdf !


In [30]:
for i in range(len(en_sentences_dict)):
    en_sentences_dict[str(i+1)] = normalizeString(en_sentences_dict[str(i+1)].strip()).strip()

In [31]:
def evaluateAll(model, test_sentences_dict):
    print(len(test_sentences_dict))
    i=0
#     j=0
    test_file = open('test_result','w')
    for i in range(len(test_sentences_dict)):
        print(i)
        out_sentence = evaluate(model, test_sentences_dict[str(i+1)]).strip() + "\n"
        out_sentence = re.sub('UNK', '', out_sentence)
        out_sentence = re.sub(' ', '', out_sentence)
        test_file.write(out_sentence)
    test_file.close()

In [48]:
evaluateAll(nmtmodel, en_sentences_dict)

8000
0
所以 ， 在 这里 ， 我 想 我 不会 和 他 在 一起 。 
1
你 是 个 大 的 体育 人 在 你 的 梦想 中 
2
距 UNK ， UNK ， UNK ， UNK 的 UNK ， 和 餐厅 的 商场 和 餐厅 。 
3
你 不 知道 我们 的 位置 ， 你 会 吗 ？ 
4
所以 我们 现在 可以 继续 把 我们 的 名单 包括 下载 谷歌 。 
5
他 得 这么 噪音 吗 ？ 我 只是 吃 了 ， 为了 上帝 ！ 
6
上尉 被 杀 了 ， 我们 还 活着 ？ 
7
别 这么 傻 。 我们 在 墨西哥 。 你 是 非法 的 。 
8
当 我 看到 你 的 儿子 ， 
9
你 两个 知道 这个 实验 吗 ？ 
10
是 啊 ， 这 不是 婴儿 可以 帮 公寓 的 。 
11
我 以前 在 这里 和 孩子 一样 。 
12
我 是 说 ， 你 以前 做 过 些 事 ... 
13
她 的 对手 和 她 的 对手 的 对手 ... 
14
但 我 觉得 我们 知道 的 是 钱 ， 不是 吗 ？ 
15
因为 我 是 自私 的 我 想 让 我 继续 ， 
16
好 吧 。 那 是 警报 。 信号 必须 启动 。 
17
我 觉得 我们 不 觉得 是 对 ， 或者 我们 能 马上 结束 。 
18
但 我们 只 看到 他 在 酒吧 里 ， 你 知道 ， 喝酒 喝 了 。 
19
你 没有 任何 信息 你 忘 了 吗 ？ 
20
我 想 你 每天 每天 都 会 痛苦 。 
21
我们 两个 两个 的 电视 ， 闹钟 ， 闹钟 、 室内 、 室内 、 安全 和 连接 。 
22
离 所有 人 。 只有 其他人 一样 的 人 。 只有 你 唯一 的 人 
23
噢 ， 那 是 好 的 。 你 说 的 是 我们 。 
24
你 的 手 是 个 。 他 被 训练 了 。 
25
她 打电话 来 了 。 我 没 回家 。 我 的 机器 在 我 的 机器 上 。 
26
我们 只 在 几周 里 ， 但 你 知道 吗 ？ 
27
不管 我们 之间 的 事 都 不 重要 。 
28
当然 ， 除非 你 的 车 会 把 你 的 车 ， 显然 。 
29
如果 你 告诉 其他人 我们 在 UNK ， 我们 可以 做到 的 。 
30
在 你 的 桌子 里 有 一个 好 

好 吧 。 你 没 把 牛奶 放 上 ， 是 吧 ？ 
260
如果 有人 发现 我 也许 会 去 上学 。 
261
但 我 需要 一些 时间 去 。 
262
我 要 做 100 次 。 我 答应 了 我 妈妈 。 
263
她 很快 会 放弃 她 的 时候 ， 她 很快 就 会 开始 。 
264
你 见 过 他 吗 ？ 他 想 打 你 电话 吗 ？ 
265
抱歉 ， 我 没 听到 你 。 我 在 洗澡 里 。 
266
所以 请 你 出去 是因为 你 不是 失败者 。 
267
希望 她 是 单身 ， 因为 我 不 做 。 
268
UNK 学生 和 学生 和 学生 的 学生 。 
269
我们 的 早餐 提供 早餐 ， 我们 的 自助 早餐 ， 我们 的 免费 服务 。 
270
把 一个 知识 都 够 了 。 但 要 告诉 ... 
271
继续 继续 进入 UNK ， 使 UNK 进入 UNK 。 
272
如果 你 想 说 的话 ， 别 把 它 从 你 包 里 。 说 。 
273
希望 那 是 个 小时 的 人 的 情况 已经 被 释放 了 。 
274
我们 的 意思 是 ， 这 是 我们 的 意思 ， 这 是 金钱 的 ， 愚蠢 的 。 
275
当 我 说 我 要 找到 你 ， 他们 把 我 留在 后面 。 
276
距 ， 提供 服务 ， 全套 服务 ， 营业 ， 营业 ， 营业 ， 晚餐 。 
277
她 的 头 会 把 窗户 从 窗户 里 。 
278
我 不想 我 丈夫 或者 我 的 孩子 。 
279
我 在 说 ， 孩子 的 事 ， 现在 没有 任何 事 。 
280
很 尴尬 的 时候 ， 婚姻 的 婚姻 们 尝试 了 离婚 。 
281
我 不能 和 我 的 脸 和 我 的 脸 不 知道 我 的 脸 。 
282
我 为什么 要 给 我 的 工作 教 一个 女孩 ？ 
283
在 早晨 ， 我们 的 早餐 ， 免费 水果 ， 每天 ， 自助 ， 和 水果 和 早餐 和 早餐 和 
284
是 啊 ， 如果 你 问 我 。 但 我 肯定 喜欢 听 起来 。 
285
别 这样 ！ 我 来 了 ， 亲爱 的 ！ 我 有 东西 。 
286
我 不 知道 你 在 说 谁 ， 但 谁 会 出去 ！ 
287
这 是 UNK 。 这 

你 的 证词 是 你 把 你 的 事业 都 在 ... 
516
宠物 允许 客人 的 客人 负责 宠物 的 酒店 。 
517
公寓 有 一间 阳台 ， 厨房 、 厨房 、 厨房 、 浴室 、 浴室 、 浴室 和 淋浴 。 
518
你 只要 救 了 我 的 屁股 ， 你 需要 做 的 。 
519
她 说 她 看到 我 的 脸 。 
520
我 没 带 你 去 参加 的 ， 队长 。 
521
是 的 ， 但 我 把 它 都 UNK 了 。 
522
他 的 身体 温度 在 尝试 和 快速 快速 地 提高 。 
523
但 我 想 我 觉得 这 是 我 想 的 。 
524
如果 你 把 舌头 的 舌头 UNK ， 
525
我们 现在 可以 住 在 我们 的 凶手 。 
526
今晚 的 船 是 什么 ？ 你 的 水 呢 ？ 
527
所以 他们 只 得到 了 100 小时 的 时间 ！ 
528
每天 在 10 点 10 日 10 00 。 
529
我 失去 了 每 一个 小时 的 时间 都 失去 了 。 
530
但 他 说 你 的 技巧 比 他 的 勇气 更 有 他 的 能力 。 
531
所以 你 想 我们 去 了 那些 的 部门 然后 把 照片 拉 出来 。 
532
你 认为 所有 的 UNK 被 爆炸 的 物质 里 ？ 
533
我 不 希望 我 能 解释 
534
为了 你 的 歌 ， 我 想 唱 一首 歌 。 
535
我 是 个 女人 。 我 不 需要 你 去 好莱坞 。 
536
你 不会 再 和 其他 的 直接 联系 。 
537
我 做 了 ， 我 不 知道 我 为什么 做 的 。 
538
好 吧 ， 打 我 。 你 想 说 什么 ？ 
539
把 他 的 工作 留在 了 一个 UNK ， 去 了 山 。 
540
我 要 给 你 吗 ？ 我 要 把 你 的 机会 给 我 。 
541
我 到处 都 到处 都 得 回家 。 
542
你 不 觉得 这 一年 会 有 选举 吗 ？ 
543
所以 在 这个 地区 的 时候 ， 在 这 片 区域 里 。 
544
噢 ， 我 在 上 面试 里 的 是 你 的 ？ 
545
他 在 哪 ？ 他 在 哪里 ？ 你 也 很 可爱 。 
546
如果 他 我 也 会 让 他们 出去 ， 太 。

你 在 那里 ， 把 我 的 UNK 卖 了 。 
773
所以 我 想 我们 想 去 去 非洲 的 时候 。 
774
我 以为 你 知道 。 你 孩子 们 怎么 了 ？ 
775
你 刚 告诉 我 我 说 的 事实 
776
只有 一个 人 在 这个 房间 里 ， 他 必须 感觉 到 他 必须 感到 。 
777
这家 酒店 酒店 的 酒店 也 是 UNK 。 
778
在 UNK 的 房间 里 ， 最后 一个 有 一只 UNK 的 鸡蛋 。 
779
但是 ， 呃 ， 你 的 新 实验室 显示 你 是 在 UNK 的 。 
780
天 啊 ， 你 知道 ， 这 只是 个 好 女友 。 
781
但 我 不想 你 这么 想 让 你 觉得 你 觉得 很 好 。 
782
你 真是 个 坚强 的 人 去 找 你 ？ 
783
听 着 ， 我们 要 去 加拿大 ， 我 叫 他们 去 哪里 ， 我 叫 他们 把 车 藏 在 哪里 。 
784
你 能 不能 被 认为 
785
当 你 想想 ， 如果 你 想 做 的 是 ， 我们 会 做 的 事 。 
786
你 说 你 没 看见 。 但 谁 会 相信 你 ？ 谁 ？ 
787
当 我们 在 我们 做 的 时候 他 就 没 在 房间 里 。 
788
我 可以 告诉 你 每个 的 每个 的 。 
789
从 我 接受 的 时候 ， 就 会 变得 更 快 。 
790
它 肯定 不 知道 ， 那 就 像 这样 一样 。 
791
如果 它 把 我们 的 所有 的 UNK 都 会 把 我们 弄 成 什么 ？ 
792
嗯 ， 我 觉得 我 觉得 好多 。 
793
如果 那些 人 的 时候 ... 然后 我们 就 坐在 那 。 
794
我 在 洛杉矶 的 现场 ， 在 洛杉矶 的 现场 
795
UNK 和 一个 男人 的 人 和 一个 人 隐瞒 的 人 。 
796
如果 你 是 真正 的 朋友 ， 你 会 理解 的 。 
797
你 知道 ， 你 知道 这 派对 的 人 都 会 知道 
798
我们 只 需 工作 
799
你 知道 我 爱 你 ， 我 永远 都 会 结婚 。 
800
噢 ， 我 ， 呃 ， 在 地下室 里 发现 了 一些 奇怪 的 俄罗斯 。 
801
他 写 了 100 本书 ， 包括 他 的 U

先生 ， 我 有 你 的 尊重 
1028
我 觉得 我 不 可能 会 说话 。 
1029
唯一 的 地图 只 显示 了 一个 敌人 的 敌人 。 
1030
她 能 唱 。 你 知道 吗 ？ 她 可以 唱 得 很 好 。 
1031
你 应该 对 我 做 同样 的 事 。 对 吧 ？ 
1032
今天 我 在 一个 特别 特殊 的 客人 
1033
我 不想 再 玩 这个 了 。 这 是 愚蠢 的 。 
1034
你 得 紧张 的 时候 我 都 会 很 紧张 。 
1035
如果 我们 想 在 UNK 的 事 里 
1036
你 知道 ， 我 不想 在 这 谈 这个 。 
1037
因为 它 不 在 那 ， 但 现在 你 想 把 它 拿 下来 ？ 
1038
如果 不是 为了 你 ， 我 就 不会 知道 外星人 存在 的 。 
1039
我 在 想 和 我 的 冰 上 ， 想 去 抓住 石头 。 
1040
他 在 6 年前 ， 他 相信 在 新鲜 的 时候 ， 他 相信 在 新鲜 的 时候 ？ 
1041
我 在 UNK 上 的 飞机 上 。 
1042
他 是 在 这里 做 的 是 UNK ！ 
1043
我们 想 和 你们 谈谈 。 
1044
但 我 想 试试 。 我 是 说 ， 我 也 想 试试 。 
1045
我 给 你 一个 军队 然后 就 会 知道 力量 的 力量 。 
1046
我 以前 经常 在 浪费时间 ， 
1047
噢 ， 拜托 。 你 一定 知道 她 都 在 你 身上 。 
1048
后来 事情 开始 去 去 去 找 她 的 行为 ... 
1049
是 我 的 朋友 或者 是 我 的 朋友 。 
1050
每个 酒店 都 被 治疗 ， 每天 吃 早餐 。 
1051
你 必须 告诉 人们 的 事 都 是 在 为 我 的 份上 。 
1052
所以 你 可能 会 有 非常 有用 的 信息 。 
1053
伙计 ， 我 不 知道 在 乐队 里 会 很 艰难 。 
1054
你 为什么 叫 我 打电话 ？ 
1055
我 现在 吃 奶酪 。 我们 能 停下来 吗 ？ 
1056
但 和 一个 用 的 UNK 和 一个 用 的 UNK 的 UNK 
1057
我 朋友 比 你 的 朋友 还 在 我 爸 身边 。 
1058
别 让 我 把 

...

如果 他 试图 让 我们 振作起来 ， 我们 杀 了 他 ， UNK ！ 
6982
你 说 我 在 这里 等 了 多久 ？ 我 来 了 10 分钟 。 
6983
如果 我 在 街上 走 ， 如果 我 到 街上 走 。 
6984
我 想 告诉 德国人 的 屁股 。 
6985
很多 人 抱怨 的 人 都 会 把 他们 的 工作 都 在 一起 。 
6986
我 就 会 把 你 的 该死 的 朋友 都 会 把 你 的 朋友 打 了 ！ 
6987
提供 UNK 的 选项 ， 使用 UNK ， 使用 UNK ， UNK UNK 的 UNK 是 UNK 。 
6988
如果 我 想 你 喝 的话 ， 我 不会 给 你 的 。 
6989
我们 能 把 那 张 ， 在 那 ， 我们 晚上 晚上 ... 
6990
如果 你 只要 听 着 ， 我们 就 会 在 5 分钟 内 睡着 。 
6991
唐 说 他 在 等 着 你 在 UNK 的 律师 等 着 你 。 
6992
我 想要 在 军队 的 任务 ， 然后 ， 在 海军 中 
6993
这些 客房 配有 两张 大床 床 、 无障碍 、 高速 互联网 接入 。 
6994
但 现在 他们 说 我 不会 在 5 30 左右 。 
6995
我 把 你 从 床上 的 床上 ， UNK 。 
6996
是 啊 ， 你 最好 穿 ， 因为 你 要 把 它 弄 开 。 
6997
在 房子 的 UNK ， 我们 的 UNK 。 
6998
我 在 车站 的 时候 ， 只有 这样 的 时候 。 
6999
他 说 了 很多 话 你 觉得 你 觉得 这 是 爱 ？ 
7000
他们 在 天空 中 ， 他们 在 天空 中 的 天空 。 
7001
我 觉得 你 不能 ... UNK 。 
7002
我 完全 在 我 的 时候 我 就 去 了 
7003
如果 她 想 改变 她 的 头发 。 
7004
在 过去 的 速度 下 ， 超过 超过 超过 超过 超过 的 速度 ， 甚至 比 每 的 速度 都 快 。 
7005
他 把 我 扔 了 UNK 。 
7006
我 能 问 你 为什么 你 在 那 时候 的 车 
7007
是 啊 ， 我 得 把 它 UNK 。 
7008
我 同意 这里 来 讨论 
7009
抱歉 ， 你 在 这 

一旦 你 把 UNK 的 UNK ， 你 将 在 UNK 。 
7228
只是 没 去过 UNK 。 这 只是 个 大 笑话 ？ 
7229
我 知道 你 不想 听 我 。 
7230
去 了 ... 从 我们 的 会议室 去 了 。 
7231
宝贝 ， 我 知道 这 很 艰难 。 我 保证 你 会 好 起来 。 
7232
你 觉得 你 不能 爱 你 的 王后 ， 你 也 不能 爱 你 吗 ？ 
7233
我 不 喜欢 他 的 手 。 
7234
我们 从 石头 的 年龄 和 社会 的 时候 。 
7235
你 要 准备 睡觉 ， 吃 个 澡 ， 就 会 看电视 。 
7236
她 有 儿子 ， 他 在 上课 。 你 知道 吗 ？ 
7237
我 想 那些 时候 的 时候 ， 我 就 想 去 学校 。 
7238
酒店 和 室外 的 室外 ， 可 像 我们 的 舒适 舒适 的 舒适 设施 。 
7239
每天 的 时候 ， 每 一天 都 会 让 我 意识 到 了 ， UNK 和 UNK 。 
7240
所以 ， 你 想 帮 你 写 一下 ， 我 要 你 写 个 论文 。 
7241
是 啊 ， 但 如果 我们 不 认为 他们 会 被 人 的 人 都 是 怪物 。 
7242
我 永远 都 不 忘记 那些 法庭 的 法律 
7243
我 觉得 我 现在 站 在 3 号 。 
7244
我 想 一个 人 的 午餐 
7245
我们 在 结婚 因为 我们 有 新 宝宝 ， 孩子 。 
7246
另外 ， 但 他们 又 在 最后 的 最后 一个 是 他们 最大 的 对手 。 
7247
他 在 学校 后 ， 我 来 和 他 谈谈 。 
7248
但 从 我 来说 ， 就 像 你 在 我 的 时候 ， 就 像 你 在 我 的 气 ， 因为 你 没有 我 。 
7249
你 在 想 我 的 UNK 在 你 的 床上 ？ 
7250
你 能 去 去 汽车旅馆 的 时候 
7251
法官 ， 在 那里 的 时候 ， 在 那里 ， 
7252
嘿 ， 嘿 ， 等等 ！ 等 ！ 
7253
你 已经 欠 了 我 一个 钱 。 他 会 给 你 一个 。 
7254
因为 你 不能 走进 我们 的 视线 。 
7255
我 是 说 ， 这 可以 让 它 做 一个 服务 。 
7256


她 第一次 想 她 的 时候 ， 她 以为 她 会 死 。 
7475
现在 开始 你 的 切断 和 UNK 的 UNK 和 记住 不 一样 。 
7476
只是 为了 好玩 ， 但 她 为什么 要 杀 猫 ？ 
7477
你 让 他 去 12 号 小组 ， 
7478
我 要 把 你 的 人 都 在 晚上 的 事 。 
7479
我 想 买 时间 ， 你 就 在 找 宝藏 。 
7480
他 很 很酷 。 你 和 你 的 阿拉伯 天使 ！ 
7481
她 病 了 ！ 如果 我们 在 同一个 公司 工作 
7482
在 那 UNK 的 时候 就 会 有 一只 。 
7483
是 啊 ， 我 从没 见 过 她 。 她 死 前 我 就 死 了 。 
7484
我 不 在 考虑 约翰 的 时间 考虑 的 是 更 多 的 。 
7485
我 肯定 能 找到 别人 为 你 照顾 。 
7486
但 你 在 高中 里 都 有 其他 女孩 。 
7487
你 需要 他们 给 我 的 提议 。 
7488
如果 你 想 相信 他 是 你 儿子 的 事 ？ 
7489
我 从没 想 过 机会 带走 她 ， 你 知道 吗 ？ 
7490
距 美式 ， 美式 、 美式 、 美式 、 营业 、 营业 、 午餐 、 午餐 、 
7491
但 我 不能 说 你 的 其他人 ！ 
7492
我 要 学 些 屁股 ？ 
7493
他 第一次 毕业 时 ， 他 ... 他 工作 了 。 
7494
如果 我 有 疫苗 ， 会 增加 疫苗 ， 
7495
嫌犯 ， 国王 ， 愿 被 惩罚 。 
7496
我 不 知道 我 的 论文 直到 我 的 论文 
7497
当然 。 但 在 地狱 里 的 UNK ， 
7498
我 对 你 来说 ， 我 很 希望 你 能 吓 你 。 
7499
UNK ， 这 是 UNK 。 走 吧 ， 狗 。 
7500
因为 你 是 这个 房间 最 重要 的 人 。 
7501
我 做 的 那种 广告 ， 我 做 了 那 是 被 她 的 UNK 。 
7502
我 把 我 的 朋友 告诉 他 去 了 。 
7503
但 我 没 给 你 。 我 是 说 。 
7504
我 知道 你 在 外面 ！ 我 知道 你 能 听到 我 ！ 
7505
从 十月 6 月 ， 十月 ， 20

可爱 的 孩子 。 聪明 的 孩子 。 这 是 他 唯一 想要 的 。 
7720
我 知道 吗 ， 妈妈 ？ 我 得 走 了 我 的 UNK 。 
7721
他 呢 ？ 承认 吧 ， 你 在 他家 。 
7722
呃 ， 先生 ， 我 想 我 已经 搞定 了 。 但 你 还有 证据 
7723
把 你 放 起来 ？ 这 是 谁 ？ 
7724
是 的 ， 我 告诉 过 你 。 不 知道 我 是 谁 。 
7725
看上去 像 舞台 上 的 舞台 。 你 有 什么 时候 吗 ？ 
7726
如果 你 不介意 我 的话 ， 你 能 让 她 做 这个 ？ 
7727
你 知道 吗 ？ 是 的 ， 我们 不 知道 他 怎么 能 把 它 弄 下来 。 
7728
如果 你 这么 想 相信 自己 的 行为 或者 反对 ， 
7729
如果 你 想 和 别人 一起 去 吗 ？ 
7730
为什么 你 不 接电话 手机 和 他 的 手机 。 
7731
我 告诉 过 你 不要 吃 什么 ？ 
7732
16 英寸 ， 配有 32 英寸 平板电视 和 32 英寸 ， 和 办公桌 和 椅子 。 
7733
是 的 。 哦 ， 当然 你 应该 休息 一下 ， 
7734
现在 你 妈妈 和 你 爸爸 说 她 去 了 
7735
所以 我们 在 飞行 中 可以 花 几个 小时 去 把 UNK 回来 。 
7736
拜托 ， 别 让 我 把 你 带到 这 东西 。 
7737
在 UNK ， UNK ， UNK ， UNK ， UNK ， UNK 。 
7738
我 很 难过 ！ 然后 我 最后 的 时候 我 就 很 难过 。 
7739
我 可 不想 再 重复 了 。 
7740
因为 如果 我 觉得 我 的 意思 是 我 想 ... 
7741
我 不想 离开 这里 ... 忘 了 我 。 
7742
你 已经 把 我 的 痛苦 都 给 了 我 的 痛苦 。 
7743
但 我 想 感谢 你 的 努力 工作 。 
7744
但 他们 不 应该 担心 爸爸 。 
7745
想 找出 谁 的 人 在 这里 的 人 受伤 。 
7746
当局 的 UNK 和 云 的 UNK 。 
7747
我 妈妈 长大 ， 我 没有 人 去 看看 ， 
7748
如果 我们 有 更 多 的 意见 。 
7

我 第一次 在 我 妈妈 的 血 里 ， 
7966
那 不 可能 的 手 可以 动 ， 所以 很 有 它 ， 
7967
如果 是 一个 人 ， 甚至 是 一个 人 ， 甚至 甚至 是 你 的 ... 
7968
下周 ， 你 就 想 我 。 
7969
我 需要 一个 人 信任 我 能 信任 
7970
我 要 去 研究 。 你 没有 研究 。 
7971
他 的 力量 比 金钱 的 力量 ， 或者 死亡 
7972
他 说 那 是 什么 电话 要 ... 
7973
但 告诉 我 你 会 杀 一个 女人 的 女人 ， 是 吧 ？ 
7974
是 那个 书 的 书 ？ 很 好 ， 是 啊 。 
7975
你 在 为 我们 的 人 更 多 的 人 。 
7976
但 这 是 在 在 做 20 年 的 时间 。 
7977
但 我 不能 改变 我 不能 从 证据 上 证明 。 
7978
这 情况 的 情况 下 ， 你 的 大脑 会 增加 你 的 能力 。 
7979
抱歉 ， 他 不 在 。 我 有 法律 问题 。 
7980
小 小 的 小 小 的 小 小 。 这里 有 一个 UNK 。 
7981
嗨 。 你 介意 在 想 和 他 的 眼睛 。 
7982
但是 在 圣诞节 ， 我 爸爸 还 没 开始 过 ， 
7983
你 觉得 她 男朋友 和 女朋友 ， 她 可以 相信 我 。 
7984
好 吧 ， 那 ... 这 是 问题 。 哦 。 
7985
因为 我们 在 吃 的 时候 我们 都 知道 什么 。 
7986
我 再也 没有 精力 要 做 这些 事 。 
7987
你 知道 你 得 走 了 吗 ？ 我 不 知道 。 
7988
如果 你 能 让 他 出去 。 
7989
我 确定 。 我们 会 做 的 。 嗯 。 
7990
离开 了 城市 的 环境 和 我 在 找 的 地方 
7991
所有 的 UNK 必须 被 UNK UNK ， UNK UNK UNK UNK UNK 。 
7992
你 问 我 如果 我 是 个 坏人 还是 个 坏蛋 ？ 
7993
把 它 放在 那里 。 然后 把 衣服 放 起来 。 
7994
我 的 目标 是 我 在 UNK 的 时候 ， 我 想 在 我 猜 。 
7995
我 知道 你 不想 再 谈 。 
7996
